# Generative Adversarial Network

生成对抗网络包含了两个子网络：生成网络(Generator，简称G)和判别网络
(Discriminator，简称D)，其中生成网络G 负责学习样本的真实分布，判别网络D 负责将
生成网络采样的样本与真实样本区分开来。

https://www.cnblogs.com/wt869054461/p/7156397.html

In [ ]:
import os
import glob
import time
import cv2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

%matplotlib inline

gpus = tf.config.experimental.list_physical_devices('GPU')
try:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
except RuntimeError as e:
    print(e)

## 1.数据导入和预处理

In [ ]:
from dataset import make_anime_dataset

BATCH_SIZE = 64
# img_path = glob.glob(r'/home/ulysses/workspace/AI/Deep-Learning-with-TensorFlow-book/ch13/faces/*.jpg')
img_path = glob.glob(r'D:\chrome_download\Deep-Learning-with-TensorFlow-book-master\Deep-Learning-with-TensorFlow-book-master\ch13/faces/*.jpg')
print('images num:', len(img_path))
dataset, img_shape, _ = make_anime_dataset(img_path, BATCH_SIZE, resize=64)
print(dataset, img_shape)
sample = next(iter(dataset)) # 采样
print(sample.shape, tf.reduce_max(sample).numpy(),
      tf.reduce_min(sample).numpy())
dataset = dataset.repeat(100) # 重复循环
db_iter = iter(dataset)

In [ ]:
for img in dataset.take(1):
#     img_ = ((img[0] + 1.0) * 127.5)
    plt.imshow(img[0])
    break

## 2.构建模型

### 构建生成器

In [ ]:
from tensorflow.keras import layers, Model
from tensorflow.keras.utils import plot_model

In [ ]:
def make_generator_model(z_dim):
    filters = 64
    model = tf.keras.Sequential([])
    
    model.add(layers.Reshape((1, 1, z_dim)))
    model.add(layers.ReLU())
    # 转置卷积-BN-激活函数:(b, 4, 4, 512)
    model.add(layers.Conv2DTranspose(filters=filters * 8, kernel_size=4, strides=1, padding='valid', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    
    # 转置卷积-BN-激活函数:(b, 8, 8, 256)
    model.add(layers.Conv2DTranspose(filters=filters * 4, kernel_size=4, strides=2, padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    
    # 转置卷积-BN-激活函数:(b, 16, 16, 128)
    model.add(layers.Conv2DTranspose(filters=filters * 2, kernel_size=4, strides=2, padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    
    # 转置卷积-BN-激活函数:(b, 32, 32, 64)
    model.add(layers.Conv2DTranspose(filters=filters * 1, kernel_size=4, strides=2, padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    
    # 转置卷积-BN-激活函数:(b, 64, 64, 3)
    model.add(layers.Conv2DTranspose(filters=3, kernel_size=4, strides=2, padding='same', use_bias=False, activation='tanh'))
    return model

In [ ]:
generator = make_generator_model(100)
# generator = Generator()
generator.build(input_shape=(None, 100))

In [ ]:
generator.summary()

In [ ]:
plot_model(generator, show_shapes=True)

In [ ]:
noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)

plt.imshow(generated_image[0, :, :, 0])

In [ ]:
generated_image.shape

### 构建判别器

In [ ]:
def make_discriminator_model():
    filters = 64
    model = tf.keras.Sequential([])
    
    
    # 卷积-BN-激活函数:(b, 31, 31, 64)  (64 - 4 + 1) / 2 向上取整
    model.add(layers.Conv2D(filters=filters * 1, kernel_size=4, strides=2, padding='valid', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    # 卷积-BN-激活函数:(b, 14, 14, 128)
    model.add(layers.Conv2D(filters=filters * 2, kernel_size=4, strides=2, padding='valid', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # 卷积-BN-激活函数:(b, 6, 6, 256)
    model.add(layers.Conv2D(filters=filters * 4, kernel_size=4, strides=2, padding='valid', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    
    # 卷积-BN-激活函数:(b, 4, 4, 512)
    model.add(layers.Conv2D(filters=filters * 8, kernel_size=3, strides=1, padding='valid', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    

    # 卷积-BN-激活函数:(b, 2, 2, 1024)
    model.add(layers.Conv2D(filters=filters * 16, kernel_size=3, strides=1, padding='valid', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # (b, 1024)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Flatten())
    model.add(layers.Dense(1))  # logits 输出

    return model

In [ ]:
discriminator = make_discriminator_model()
discriminator.build(input_shape=(4, 64, 64, 3))

In [ ]:
discriminator.summary()

## 3. 定义损失函数

In [ ]:
# 判别器和生成器的损失
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
def discriminator_loss(real_output, fake_output):
    # real 越接近1越好
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    # fake 越接近0越好
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return tf.reduce_mean(total_loss)

def generator_loss(fake_output):
    # 生成的数据结果越接近1越好
    loss = cross_entropy(tf.ones_like(fake_output), fake_output)
    return tf.reduce_mean(loss)

learning_rate = 2e-4
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.5)

```python3
checkpoint_directory = "/tmp/training_checkpoints"
checkpoint_prefix = os.path.join(checkpoint_directory, "ckpt")

checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=model)
status = checkpoint.restore(tf.train.latest_checkpoint(checkpoint_directory))
for _ in range(num_training_steps):
  optimizer.minimize( ... )  # Variables will be restored on creation.
status.assert_consumed()  # Optional sanity checks.
checkpoint.save(file_prefix=checkpoint_prefix)
```

In [ ]:
# checkpoint
check_point_dir = './train_checkpoints'
check_point_prefix = os.path.join(check_point_dir, 'ckpt')
check_point = tf.train.Checkpoint(generator_optimizer = generator_optimizer,
                                  discriminator_optimizer = discriminator_optimizer,
                                  generator=generator,
                                  discriminator=discriminator)

## 4. 训练函数

In [ ]:
EPOCHS = 30000
z_dim = 100 # 隐向量z维度
K = 5  #  判断器迭代次数 

nums_examples_to_generate = 100
seed = tf.random.normal([nums_examples_to_generate, z_dim])

In [ ]:
# 迭代函数
@tf.function
def train_step():
    
    noise = tf.random.normal([BATCH_SIZE, z_dim])
    images = next(db_iter)
    
    for _ in range(K):
        with tf.GradientTape() as disc_tape:
            generated_images = generator(noise, training=True)
            real_output = discriminator(images, training=True)
            fake_output = discriminator(generated_images, training=True)
            dis_loss = discriminator_loss(real_output, fake_output)

        gradients_of_discriminator = disc_tape.gradient(dis_loss, discriminator.trainable_variables)
        discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
    # 避免更新太多而令 JS 散度上升。
    with tf.GradientTape() as gen_tape:
        noise = tf.random.normal([BATCH_SIZE, z_dim])
        generated_images = generator(noise, training=True)
        fake_output = discriminator(generated_images, training=True)
        gen_loss = generator_loss(fake_output)
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    
    return gen_loss, dis_loss

生成图片

In [ ]:
def generate_and_save_images(model, val_block_size, image_path, seed):
    def preprocess(img):
        img = ((img + 1.0) * 127.5).astype(np.uint8)
        # img = img.astype(np.uint8)
        return img
    val_out = model(seed, training=False)
    preprocesed = preprocess(val_out.numpy())
    final_image = np.array([])
    single_row = np.array([])
    for b in range(val_out.shape[0]):
        # concat image into a row
        if single_row.size == 0:
            single_row = preprocesed[b, :, :, :]
        else:
            single_row = np.concatenate((single_row, preprocesed[b, :, :, :]), axis=1)

        # concat image row to final_image
        if (b+1) % val_block_size == 0:
            if final_image.size == 0:
                final_image = single_row
            else:
                final_image = np.concatenate((final_image, single_row), axis=0)

            # reset single row
            single_row = np.array([])

    if final_image.shape[2] == 1:
        final_image = np.squeeze(final_image, axis=2)
    # toimage(final_image).save(image_path)
    # io.imsave(image_path, final_image)
    cv2.imwrite(image_path, final_image)

In [ ]:
def train():
    d_losses, g_losses = [],[]
    for step in range(EPOCHS):
        start = time.time()
        gen_loss, dis_loss = train_step()
        if step%100 == 0:
            print(step, 'd-loss:',float(dis_loss), 'g-loss:', float(gen_loss))
            d_losses.append(dis_loss)
            g_losses.append(gen_loss)
            # 查看同一个seed 生成的不同图片
            img_path = os.path.join('gan_images1', 'gan-%d.png'%step)
            generate_and_save_images(generator, 10, img_path, seed)
        if (step + 1) % 10000 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)
    img_path = os.path.join('gan_images1', 'gan-%d.png'%step)
    generate_and_save_images(generator, 10, img_path, seed)
    return  d_losses, g_losses  

In [ ]:
d_losses, g_losses = train()